# Data Preprocessing

Notebook dùng để tiền xử lý dữ liệu: Load, làm sạch, lấy mẫu (sampling), và gộp các bảng dữ liệu.

In [ ]:
import pandas as pd
import os

## 1. Cấu hình và Đường dẫn
Khai báo đường dẫn đến các file dữ liệu.

In [ ]:
from pathlib import Path
base_path = Path('rawdata')

# Fallback to 'archive' if 'rawdata' is not present
if not base_path.exists():
    alt = Path('archive')
    if alt.exists():
        base_path = alt
    else:
        raise FileNotFoundError(f"Neither 'rawdata' nor 'archive' directories exist in {Path.cwd()}")

# Paths
orders_path = base_path / 'Orders.csv'
details_path = base_path / 'Order_Details.csv'
customers_path = base_path / 'Customers.csv'

print(f"Using data directory: {base_path}")

## 2. Đọc dữ liệu
Đọc dữ liệu từ file CSV

In [ ]:
print("Loading data...")

# Helper to try different encodings/separators
def robust_read_csv(path, **kwargs):
    try:
        return pd.read_csv(path, **kwargs)
    except Exception:
        # Try common alternatives
        for sep in [';', ',']:
            for enc in ['utf-8', 'latin-1']:
                try:
                    return pd.read_csv(path, sep=sep, encoding=enc, engine='python')
                except Exception:
                    continue
        raise

# Load Orders
if not orders_path.exists():
    raise FileNotFoundError(f"Orders file not found at: {orders_path}")
orders = robust_read_csv(orders_path)
print(f"Orders shape: {orders.shape}")

# Load Details
if not details_path.exists():
    raise FileNotFoundError(f"Order details file not found at: {details_path}")
details = robust_read_csv(details_path)
print(f"Details shape: {details.shape}")

# Load Customers
if not customers_path.exists():
    raise FileNotFoundError(f"Customers file not found at: {customers_path}")
customers = robust_read_csv(customers_path)
print(f"Customers shape: {customers.shape}")

## 3. Lấy mẫu (Sampling)
Giảm kích thước dữ liệu để thuận tiện cho việc xử lý và phân tích. Chúng ta lấy mẫu theo Orders ID để đảm bảo tính toàn vẹn của đơn hàng.

In [ ]:
print("Sampling orders to reduce dataset size...")
# Target: Xử lý tập con khoảng 50,000 - 60,000 dòng chi tiết.
# Ước tính mỗi đơn hàng có khoảng 5 item -> Cần khoảng 10,000 - 12,000 đơn hàng.

if len(orders) > 12000:
    orders = orders.sample(n=12000, random_state=42)
print(f"Sampled Orders shape: {orders.shape}")

## 4. Làm sạch dữ liệu (Data Cleaning)
- Chuyển đổi định dạng tiền tệ (chuỗi có dấu phẩy sang float)
- Chuyển đổi định dạng ngày tháng
- Lọc dữ liệu chi tiết và khách hàng theo danh sách đơn hàng đã lấy mẫu

In [ ]:
print("Cleaning data...")

# Helper to clean currency/float strings with commas
import numpy as np

def clean_currency(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, str):
        return float(x.replace(',', '.'))
    try:
        return float(x)
    except Exception:
        return np.nan

# Clean Orders
if 'TOTALBASKET' in orders.columns:
    orders['TOTALBASKET'] = orders['TOTALBASKET'].apply(clean_currency)
else:
    print('Warning: TOTALBASKET not found in Orders')

orders['DATE_'] = pd.to_datetime(orders['DATE_'], errors='coerce')

# Clean Details - filter by sampled orders first
print("Filtering details...")
details = details[details['ORDERID'].isin(orders['ORDERID'])]
print(f"Filtered Details shape: {details.shape}")

cols_to_clean = ['UNITPRICE', 'TOTALPRICE']
for col in cols_to_clean:
    if col in details.columns:
        details.loc[:, col] = details[col].apply(clean_currency)
    else:
        print(f"Warning: {col} not found in details")
    
# Clean Customers
customers = customers[customers['USERID'].isin(orders['USERID'])]
customers['USERBIRTHDATE'] = pd.to_datetime(customers['USERBIRTHDATE'], errors='coerce')

# Fail early if key columns missing
required = ['ORDERID','USERID']
for dfname, df in [('orders', orders), ('details', details), ('customers', customers)]:
    for col in required:
        if col not in df.columns:
            raise KeyError(f"Required column {col} missing in {dfname} dataframe")

## 5. Hợp nhất dữ liệu (Merging)
Kết nối 3 bảng Orders, Order Details, và Customers lại thành một bảng duy nhất.

In [ ]:
print("Merging data...")

# Merge Orders with Details on ORDERID
merged_orders = pd.merge(details, orders, on='ORDERID', how='left')

# Merge with Customers on USERID
final_df = pd.merge(merged_orders, customers, on='USERID', how='left')
print(f"Merged shape before final cut: {final_df.shape}")

# Final downsample: Đảm bảo kích thước file không quá lớn (giữ nguyên logic kiểm tra > 1M, 
# nhưng thực tế với 12k đơn hàng thì số dòng thường chỉ khoảng 60k)
if len(final_df) > 1000000:
    print(f"Downsampling from {len(final_df)} to sample...")
    final_df = final_df.sample(n=1000000, random_state=42)

print(f"Final merged shape: {final_df.shape}")

## 6. Feature Engineering
Tạo các đặc trưng mới:
- **Year, Month**: Từ ngày đặt hàng
- **Age**: Tuổi của khách hàng

In [ ]:
# Ensure DATE_ is datetime
final_df['Year'] = final_df['DATE_'].dt.year
final_df['Month'] = final_df['DATE_'].dt.month

# Calculate Age safely
current_year = pd.Timestamp.now().year
final_df['Age'] = final_df['USERBIRTHDATE'].dt.year.apply(lambda y: current_year - y if pd.notna(y) else pd.NA)

## 7. Lưu và Kiểm tra kết quả
Lưu dữ liệu đã xử lý ra file CSV mới và hiển thị thông tin mẫu.

In [ ]:
output_dir = Path('data')
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / 'processed_data.csv'
final_df.to_csv(output_path, index=False)
print(f"Saved processed data to {output_path}")

# Display info
print(final_df.info())
final_df.head()